Process raw AQI data downloaded from EPA website, extract and summarize columns for data analysis

In [26]:
import pandas as pd
import glob
import os

# Path to the directory containing your CSV files
directory_path = './raw_data/Annual_AQI' 

# Pattern to match all CSV files of interest
file_pattern = os.path.join(directory_path, 'annual_aqi_by_county_*.csv')

# List to hold data from all files
all_data = []

In [27]:
# Process each file
for file in glob.glob(file_pattern):
    df = pd.read_csv(file)
    # Calculating percentages of AQI categories
    df['% Good Days'] = df['Good Days'] / df['Days with AQI']
    df['% Moderate Days'] = df['Moderate Days'] / df['Days with AQI']
    df['% Unhealthy for Sensitive Groups Days'] = df['Unhealthy for Sensitive Groups Days'] / df['Days with AQI']
    df['% Unhealthy Days'] = df['Unhealthy Days'] / df['Days with AQI']
    df['% Very Unhealthy Days'] = df['Very Unhealthy Days'] / df['Days with AQI']
    df['% Hazardous Days'] = df['Hazardous Days'] / df['Days with AQI']
    df['% Ozone Days'] = df['Days Ozone'] / df['Days with AQI']
    df['% PM2.5 Days'] = df['Days PM2.5'] / df['Days with AQI']

    # Extract the year from the filename
    year = file[-8:-4]  # Assumes the year is always the last four characters before .csv

    # Aggregate data by state for the current year
    state_summary = df.groupby('State').agg({
        '% Good Days': 'mean',
        '% Moderate Days': 'mean',
        '% Unhealthy for Sensitive Groups Days': 'mean',
        '% Unhealthy Days': 'mean',
        '% Very Unhealthy Days': 'mean',
        '% Hazardous Days': 'mean',
        '% Ozone Days': 'mean',
        '% PM2.5 Days': 'mean',
        '90th Percentile AQI': 'mean',
        'Median AQI': 'mean'
    }).reset_index()

    # Add the year column to the state summary
    state_summary['Year'] = year

    # Append to the all_data list
    all_data.append(state_summary)

# Concatenate all dataframes
complete_data = pd.concat(all_data, ignore_index=True)

# Rearrange columns so that 'Year' is the first column
complete_data = complete_data[['Year'] + [col for col in complete_data.columns if col != 'Year']]
complete_data = complete_data.round(5)

# Save the summarized data to a new CSV file
output_path = './data_processed/summary_aqi_by_state2.csv'  
complete_data.to_csv(output_path, index=False)

print(f"Data processed and saved to {output_path}")

Data processed and saved to /Users/smiie/Desktop/RespiratoryHealth/data_processed/summary_aqi_by_state2.csv


Below code convert Asthema data extracted from CDC website in Excel into csv file

In [3]:
import pandas as pd

# Path to the Excel file
excel_path = './raw_data/CDC_Asthma.xlsx'

# Path for the output CSV file
output_csv_path = './data_processed/asthma_data.csv'

# Load the 'asthma' sheet into a DataFrame
df = pd.read_excel(excel_path, sheet_name='asthma')

df.rename(columns={'State or Territory': 'State'}, inplace=True)

# Save the DataFrame to a CSV file
df.to_csv(output_csv_path, index=False)

print(f"Data from 'asthma' sheet exported successfully to {output_csv_path}")

Data from 'asthma' sheet exported successfully to /Users/smiie/Desktop/RespiratoryHealth/data_processed/asthma_data.csv


Pair and add all data into one single csv file for data analysis

In [15]:
import pandas as pd

# Load the datasets
path_aqi = './data_processed/summary_aqi_by_state.csv'  # Update with the correct path
path_asthma = './RespiratoryHealth/data_processed/asthma_data.csv'  # Update with the correct path

# Read the CSV files
df_aqi = pd.read_csv(path_aqi)
df_asthma = pd.read_csv(path_asthma)

df_combined = pd.merge(df_aqi, df_asthma, on=['State', 'Year'], how='inner')

# Save the combined data to a single CSV file
output_path = './RespiratoryHealth/data_processed/AQI_asthma_combined_data.csv'  # Update with the correct path
df_combined.to_csv(output_path, index=False)

print(f"Combined data saved to {output_path}")

Combined data saved to /Users/smiie/Desktop/RespiratoryHealth/data_processed/AQI_asthma_combined_data.csv


Convert asthema mortality data from Excel to CSV file

In [6]:
import pandas as pd

# Path to the Excel file
excel_path = './raw_data/Asthma_mortality.xlsx'

# Path for the output CSV file
output_csv_path = './data_processed/asthma_mortality_data.csv'

# Load the 'asthma' sheet into a DataFrame
df = pd.read_excel(excel_path, sheet_name='mortality')

# Save the DataFrame to a CSV file
df.to_csv(output_csv_path, index=False)

print(f"Data from 'asthma' sheet exported successfully to {output_csv_path}")

Data from 'asthma' sheet exported successfully to /Users/smiie/Desktop/RespiratoryHealth/data_processed/asthma_mortality_data.csv


In [2]:
import pandas as pd

# Load the datasets
asthma_AQI_data_path = './data_processed/AQI_asthma_combined_data.csv'
asthma_mortality_data_path = './data_processed/asthma_mortality_data.csv'

asthma_AQI_data = pd.read_csv(asthma_AQI_data_path)
asthma_mortality_data = pd.read_csv(asthma_mortality_data_path)

# Merge the datasets on 'State' and 'Year'
combined_data = pd.merge(asthma_AQI_data, asthma_mortality_data, on=['State', 'Year'], how='inner')

# Save the combined data to a new CSV file
combined_data_output_path = './data_processed/combined_asthma_data2.csv'
combined_data.to_csv(combined_data_output_path, index=False)

print(f"Combined data saved successfully to {combined_data_output_path}")



Combined data saved successfully to ./data_processed/combined_asthma_data2.csv


In [19]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('./data_processed/asthma_data.csv')

# Dictionary to map state names to abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 
    'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 
    'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Map full names to abbreviations
df['State Abbreviation'] = df['State'].map(state_abbreviations)

# Save the updated CSV
df.to_csv('./Dashboard/data/asthma_data_map_abb.csv', index=False)


In [22]:
import pandas as pd

def merge_datasets():
    # Load the current asthma dataset
    asthma_data_path = "./Dashboard/data/asthma_data_map_abb.csv"
    asthma_data = pd.read_csv(asthma_data_path)

    # Load the mortality data
    mortality_data_path = "./data_processed/asthma_mortality_data.csv"
    mortality_data = pd.read_csv(mortality_data_path)

    # Merge the datasets on 'State Abbreviation' and 'Year'
    # Ensure both dataframes have the 'Year' column as integers if they're not
    asthma_data['Year'] = asthma_data['Year'].astype(int)
    mortality_data['Year'] = mortality_data['Year'].astype(int)

    # Merge operation
    merged_data = pd.merge(asthma_data, mortality_data, how='left', on=['State', 'Year'])
    merged_data = merged_data.reindex(columns=['State Abbreviation'] + [col for col in merged_data.columns if col != 'State Abbreviation'])

    # Save the merged dataset back to a CSV file
    output_path = "./Dashboard/data/merged_asthma_mortality_data.csv"
    merged_data.to_csv(output_path, index=False)

    print("Merged dataset saved to:", output_path)

# Call the function to perform the merge
merge_datasets()


Merged dataset saved to: /Users/smiie/Desktop/RespiratoryHealth/Dashboard/data/merged_asthma_mortality_data.csv


In [24]:
import pandas as pd

# Load the CODP mortality data
file_path_copd = "./raw_data/State_COPD_mortality.csv"
copd_data = pd.read_csv(file_path_copd)

# Load the AQI data
file_path_aqi = "./data_processed/summary_aqi_by_state.csv"
aqi_data = pd.read_csv(file_path_aqi)

# Filter both datasets for the year 2021 if necessary
copd_data = copd_data[copd_data['Year'] == 2021]
aqi_data = aqi_data[aqi_data['Year'] == 2021]

# Merge the datasets on the State column
merged_data = pd.merge(aqi_data, copd_data[['State', 'Age-Adjusted Mortality (per 100,000)']], on='State', how='left')

# Save the merged data to a new CSV file
output_path = "./data_processed/merged_aqi_copd_2021.csv"
merged_data.to_csv(output_path, index=False)

print("Merged data saved to:", output_path)



Merged data saved to: /Users/smiie/Desktop/RespiratoryHealth/data_processed/merged_aqi_copd_2021.csv


In [25]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert the Excel file to CSV
excel_path = "./raw_data/life_long_asthma.xlsx"
csv_path = "./data_processed/life_long_asthma.csv"

# Load the data from Excel
data = pd.read_excel(excel_path)

# Save as CSV
data.to_csv(csv_path, index=False)

Match CODP data with AQI

In [4]:
import pandas as pd

# Load the CODP mortality data
file_path_copd = "./raw_data/State_COPD_prevalence2022.csv"
copd_data = pd.read_csv(file_path_copd)

# Load the AQI data
file_path_aqi = "./data_processed/summary_aqi_by_state2.csv"
aqi_data = pd.read_csv(file_path_aqi)

# Filter both datasets for the year 2021 if necessary
copd_data = copd_data[copd_data['Year'] == 2022]
aqi_data = aqi_data[aqi_data['Year'] == 2022]

# Merge the datasets on the State column
merged_data = pd.merge(aqi_data, copd_data[['State', 'Age-Adjusted Prevalence (%)']], on='State', how='left')

# Save the merged data to a new CSV file
output_path = "./data_processed/merged_aqi_copd_2022.csv"
merged_data.to_csv(output_path, index=False)

print("Merged data saved to:", output_path)

Merged data saved to: ./data_processed/merged_aqi_copd_2022.csv
